## Data wrangling to reproduce Istat-based KPIs

In [ ]:
from enum import Enum
from os import path, listdir

import numpy as np
import pandas as pd
import geopandas as gpd
import geopy, geopy.distance
import shapely
from sklearn import gaussian_process

from matplotlib import pyplot as plt 
import seaborn as sns
plt.rcParams['figure.figsize']= (20,14)

In [ ]:
## TODO: find way to put this into some global settings
import os
import sys
nb_dir = os.path.dirname(os.getcwd())
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

from references import common_cfg
from references import istat_kpi

In [ ]:
selectedCity = common_cfg.cityList[1]
print(selectedCity)

In [ ]:
# load sezione di censimento data for the selected city
istatData = common_cfg.get_istat_cpa_data(selectedCity)

In [ ]:
istatData.head()

In [ ]:
if selectedCity == 'Milano':
    # groupby Nome Quartiere
    quartiereData = istatData.groupby([common_cfg.IdQuartiereColName, 'quartiere']).sum()
elif selectedCity == 'Torino':
    quartiereData = istatData.groupby([common_cfg.IdQuartiereColName]).sum()
quartiereData.head()

In [ ]:
metaData = common_cfg.get_istat_metadata()
metaData.head()

## Indici

1. Percentuale residenti stranieri.
    - Sul totale dei residenti
    
2. Percentuali di provenienza degli stranieri (*per ogni continente la percentuale è rispetto la popolazione residente straniera*).
    - Europa
    - Africa
    - Asia
    - America
    - Oceania
    
3. Indice di vecchiaia: *rapporto tra il numero di residenti di età > 64 e numero residenti che hanno meno di 14 anni.*
    - Sul totale popolazione
    - Tra gli uomini
    - Tra le donne
    
4. Indice di popolazione attiva: *rapporto tra il numero di residenti con età compresa tra i 39 ed i 64 anni e quelli tra i 15 ed i 39.*
    - Sul totale popolazione
    - Tra gli uomini
    - Tra le donne
    
5. Popolazione residente che svolge la propria giornata fuori dal comune di residenza sul totale della popolazione residente.

6. Popolazione residente che svolge la propria giornata nel comune ma fuori dal proprio quartiere

7. Popolazione residente che svolge la propria giornata nel proprio quartiere

#### 1. Percentuale residenti stranieri

In [ ]:
# Colonne ISTAT utilizzate
metaData[(metaData['NOME_CAMPO'] == 'P1') | (metaData['NOME_CAMPO'] == 'ST15')]

In [ ]:
# Creazione indice residenti stranieri
istat_kpi.new_index(quartiereData, ['ST15'], ['P1'], 'indice_stranieri').head()

#### 2. Percentuali di provenienza degli stranieri

In [ ]:
# Colonne ISTAT utilizzate
metaData[(metaData['NOME_CAMPO'] == 'ST9') | 
         (metaData['NOME_CAMPO'] == 'ST10')|
         (metaData['NOME_CAMPO'] == 'ST11')|
         (metaData['NOME_CAMPO'] == 'ST12')|
         (metaData['NOME_CAMPO'] == 'ST13')|
         (metaData['NOME_CAMPO'] == 'ST15')]

In [ ]:
# Lista di nomi di variabili che voglio creare 
list_columns_continenti = ['perc_europei', 'perc_africa', 'perc_america', 'perc_asia', 'perc_oceania']

# Lista colonne utili dal df di istat
list_attr_continenti = ['ST9', 'ST10', 'ST11', 'ST12', 'ST13']

In [ ]:
# Calcolare l'indice per ogni continente
for idx, continente in enumerate(list_columns_continenti):
    istat_kpi.new_index(quartiereData, [list_attr_continenti[idx]], ['ST15'], list_columns_continenti[idx])

In [ ]:
quartiereData.head()

#### 3. Indice vecchiaia

##### 3.1. Totale popolazione residente

In [ ]:
# Colonne ISTAT utilizzate
metaData[(metaData['NOME_CAMPO'] == 'P27') | 
         (metaData['NOME_CAMPO'] == 'P28')|
         (metaData['NOME_CAMPO'] == 'P29')|
         (metaData['NOME_CAMPO'] == 'P14')|
         (metaData['NOME_CAMPO'] == 'P15')|
         (metaData['NOME_CAMPO'] == 'P16')]

In [ ]:
# Creo la nuova colonna
istat_kpi.new_index(quartiereData, ['P27','P28','P29'], ['P14','P15','P16'], 'indice_vecchiaia').head()

##### 3.2. Totale popolazione residente maschile

In [ ]:
# Colonne ISTAT utilizzate
metaData[(metaData['NOME_CAMPO'] == 'P43') | 
         (metaData['NOME_CAMPO'] == 'P44')|
         (metaData['NOME_CAMPO'] == 'P45')|
         (metaData['NOME_CAMPO'] == 'P30')|
         (metaData['NOME_CAMPO'] == 'P31')|
         (metaData['NOME_CAMPO'] == 'P32')]

In [ ]:
# Creo la nuova colonna
istat_kpi.new_index(quartiereData, ['P43','P44','P45'], ['P30','P31','P32'], 'indice_vecchiaia_uomo').head()

##### 3.3. Totale popolazione residente femminile
Il numero della popolazione femminile non è espresso esplicitamente nel dataset. Pertanto è ottenuto facendo la differenza tra la popolazione residente totale e quella maschile.

Le colonne utilizzate sono descritte nelle due sezioni precedenti (`3.1` e `3.2`).

In [ ]:
# Calcola numeratore e denominatore
donne_anziane = quartiereData[['P27','P28','P29']].sum(axis=1)-(quartiereData[['P43','P44','P45']].sum(axis=1))
donne_giovani = quartiereData[['P14','P15','P16']].sum(axis=1)-quartiereData[['P30','P31','P32']].sum(axis=1)

# Quindi, il nuovo indice
quartiereData['indice_vecchiaia_donna'] = donne_anziane/donne_giovani

quartiereData.head()

#### 4. Indice popolazione attiva

##### 4.1. Totale popolazione residente


In [ ]:
# Colonne ISTAT utilizzate
metaData[(metaData['NOME_CAMPO'] == 'P22')| 
         (metaData['NOME_CAMPO'] == 'P23')|
         (metaData['NOME_CAMPO'] == 'P24')|
         (metaData['NOME_CAMPO'] == 'P25')|
         (metaData['NOME_CAMPO'] == 'P26')|
         (metaData['NOME_CAMPO'] == 'P17')|
         (metaData['NOME_CAMPO'] == 'P18')|
         (metaData['NOME_CAMPO'] == 'P19')|
         (metaData['NOME_CAMPO'] == 'P20')|
         (metaData['NOME_CAMPO'] == 'P21')]

In [ ]:
# Creo la nuova colonna
istat_kpi.new_index(quartiereData, ['P22','P23','P24','P25','P26'], ['P17','P18','P19','P20','P21'], 'indice_pop_attiva').head()

##### 4.2. Popolazione residente maschile

In [ ]:
# Colonne ISTAT utilizzate
metaData[(metaData['NOME_CAMPO'] == 'P38')| 
         (metaData['NOME_CAMPO'] == 'P39')|
         (metaData['NOME_CAMPO'] == 'P40')|
         (metaData['NOME_CAMPO'] == 'P41')|
         (metaData['NOME_CAMPO'] == 'P42')|
         (metaData['NOME_CAMPO'] == 'P33')|
         (metaData['NOME_CAMPO'] == 'P34')|
         (metaData['NOME_CAMPO'] == 'P35')|
         (metaData['NOME_CAMPO'] == 'P36')|
         (metaData['NOME_CAMPO'] == 'P37')]

In [ ]:
# Creo la nuova colonna
istat_kpi.new_index(quartiereData, ['P38','P39','P40','P41','P42'], ['P33','P34','P35','P36','P37'], 'indice_pop_attiva').head()

##### 4.3. Popolazione residente femminile

Anche in questo caso, i numeri sulle donne sono ricavati dalla differenza tra il totale e la popolazione maschile.

In [ ]:
# Calcolo numeratore e denominatore
numeratore_donne = (quartiereData[['P22','P23','P24','P25','P26']].sum(axis=1)) - (quartiereData[['P38','P39','P40','P41','P42']].sum(axis=1))
denominatore_donne = (quartiereData[['P17','P18','P19','P20','P21']].sum(axis=1))-(quartiereData[['P33','P34','P35','P36','P37']].sum(axis=1))

# Creazione colonna
quartiereData['indice_pop_attiva_donna'] = numeratore_donne/denominatore_donne

#### 5.  Popolazione residente che svolge la propria giornata fuori dal comune di residenza sul totale della popolazione residente

In [ ]:
# Colonne ISTAT utilizzate
metaData[(metaData['NOME_CAMPO'] == 'P138')]

In [ ]:
# Creo la nuova colonna
istat_kpi.new_index(quartiereData, ['P138'], ['P1'], 'indice_pop_pendolare').head()

#### 6.  Popolazione residente che svolge la propria giornata nel comune di residenza ma al di fuori del proprio quartiere sul totale della popolazione residente.

In [ ]:
# Colonne ISTAT utilizzate
metaData[(metaData['NOME_CAMPO'] == 'P137')]

In [ ]:
# Creo la nuova colonna
istat_kpi.new_index(quartiereData, ['P137'], ['P1'], 'indice_pop_non_pend_esterna_quartiere').head()

#### 7.  Popolazione residente che svolge la propria giornata nel proprio quartiere sul totale della popolazione residente.

Il valore assoluto della popolazione che non svolge la giornata nel proprio quartiere è pari alla differenza tra `P1` e la somma di `P137` e `P138`.

In [ ]:
giornata_dentro_quartiere = quartiereData['P1']-(quartiereData[['P137','P138']].sum(axis=1))

quartiereData['indice_pop_non_pend_interna_quartiere'] = giornata_dentro_quartiere/quartiereData['P1']

In [ ]:
quartiereData.head()

### Preview del dataset degli indici

In [ ]:
quartiereData[list(quartiereData.columns)[-11:]].head()

# Salva df in csv
quartiereData[list(quartiereData.columns)[-11:]].to_csv(
    '../data/output/'+selectedCity+ '_kpi_istat_cpa2011.csv', sep = ';')

# Salva in json
quartiereData[list(quartiereData.columns)[-11:]].reset_index().to_json(
    '../data/output/'+selectedCity+ '_kpi_istat_cpa2011.json', orient='records')
